In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error
from pprint import pprint
import pickle

In [ ]:
data_pd = pd.read_csv('data/data_AMP_PCS_reduced.csv')

In [9]:
data_pd.columns

Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object')

In [10]:
data_pd_y = data_pd.loc[:,'5':]
data_pd_X = data_pd.loc[:,:'4']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data_pd_X, data_pd_y, test_size=0.33, random_state=42, shuffle=True)

## Default Random Forest

In [8]:
regr = RandomForestRegressor(random_state=0, verbose=2)

In [9]:
regr.fit(X_train, y_train)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  8.8min


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

RandomForestRegressor(random_state=0, verbose=2)

In [10]:
predictions = regr.predict(X_test)
 
# Evaluating the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

rmse = root_mean_squared_error(y_test, predictions)
print(f'Root Mean Squared Error: {rmse}')
 
r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   48.8s


Mean Squared Error: 275137.941568655
Mean Absolute Error: 165.00234884091466
Root Mean Squared Error: 420.87244168093025
R-squared: 0.9978419972375931


In [12]:
regr.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 2,
 'warm_start': False}

In [35]:
import pickle

In [15]:
with open('Models/random-forest-default.pkl','wb') as f:
    pickle.dump(regr,f)

## Randomized search random forest

In [8]:
from sklearn.model_selection import RandomizedSearchCV

In [9]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [23]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, scoring = 'neg_root_mean_squared_error', cv = 3, verbose=2, random_state=42, n_jobs = None)# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time= 1.3min
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time= 1.3min
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time= 1.3min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 5.4min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 5.5min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 5.5min
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=6

/home/psolikov/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/psolikov/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/psolikov/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/psolikov/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  F

KeyboardInterrupt: 

In [ ]:
# import scipy
# scipy.test()


In [24]:
rf_random.best_params_

{'n_estimators': 1400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [6]:
rf_best_from_random = RandomForestRegressor(n_estimators=1000, min_samples_split=5, min_samples_leaf=1,max_depth=30, bootstrap=True, verbose=2)

In [7]:
rf_best_from_random.fit(X_train, y_train)

building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   58.4s


building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000
building tree 46 of 1000
building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000
building tree 52 of 1000
building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000
building tree 60 of 1000
building tree 61 of 1000
building tree 62 of 1000
building tree 63 of 1000
building tree 64 of 1000
building tree 65 of 1000
building tree 66 of 1000
building tree 67 of 1000
building tree 68 of 1000
building tree 69 of 1000
building tree 70 of 1000
building tree 71 of 1000
building tree 72 of 1000
building tree 73 of 1000
building tree 74 of 1000
building tree 75 of 1000
building tree 76 of 1000
building tree 77 of 1000
building tree 78 of 1000
building tree 79 of 1000
building tree 80 of 1000


[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  3.9min


building tree 162 of 1000
building tree 163 of 1000
building tree 164 of 1000
building tree 165 of 1000
building tree 166 of 1000
building tree 167 of 1000
building tree 168 of 1000
building tree 169 of 1000
building tree 170 of 1000
building tree 171 of 1000
building tree 172 of 1000
building tree 173 of 1000
building tree 174 of 1000
building tree 175 of 1000
building tree 176 of 1000
building tree 177 of 1000
building tree 178 of 1000
building tree 179 of 1000
building tree 180 of 1000
building tree 181 of 1000
building tree 182 of 1000
building tree 183 of 1000
building tree 184 of 1000
building tree 185 of 1000
building tree 186 of 1000
building tree 187 of 1000
building tree 188 of 1000
building tree 189 of 1000
building tree 190 of 1000
building tree 191 of 1000
building tree 192 of 1000
building tree 193 of 1000
building tree 194 of 1000
building tree 195 of 1000
building tree 196 of 1000
building tree 197 of 1000
building tree 198 of 1000
building tree 199 of 1000
building tre

[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  8.7min


building tree 365 of 1000
building tree 366 of 1000
building tree 367 of 1000
building tree 368 of 1000
building tree 369 of 1000
building tree 370 of 1000
building tree 371 of 1000
building tree 372 of 1000
building tree 373 of 1000
building tree 374 of 1000
building tree 375 of 1000
building tree 376 of 1000
building tree 377 of 1000
building tree 378 of 1000
building tree 379 of 1000
building tree 380 of 1000
building tree 381 of 1000
building tree 382 of 1000
building tree 383 of 1000
building tree 384 of 1000
building tree 385 of 1000
building tree 386 of 1000
building tree 387 of 1000
building tree 388 of 1000
building tree 389 of 1000
building tree 390 of 1000
building tree 391 of 1000
building tree 392 of 1000
building tree 393 of 1000
building tree 394 of 1000
building tree 395 of 1000
building tree 396 of 1000
building tree 397 of 1000
building tree 398 of 1000
building tree 399 of 1000
building tree 400 of 1000
building tree 401 of 1000
building tree 402 of 1000
building tre

[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed: 15.5min


building tree 648 of 1000
building tree 649 of 1000
building tree 650 of 1000
building tree 651 of 1000
building tree 652 of 1000
building tree 653 of 1000
building tree 654 of 1000
building tree 655 of 1000
building tree 656 of 1000
building tree 657 of 1000
building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000
building tree 674 of 1000
building tree 675 of 1000
building tree 676 of 1000
building tree 677 of 1000
building tree 678 of 1000
building tree 679 of 1000
building tree 680 of 1000
building tree 681 of 1000
building tree 682 of 1000
building tree 683 of 1000
building tree 684 of 1000
building tree 685 of 1000
building tre

RandomForestRegressor(max_depth=30, min_samples_split=5, n_estimators=1000,
                      verbose=2)

In [8]:
predictions = rf_best_from_random.predict(X_test)
 
# Evaluating the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

rmse = root_mean_squared_error(y_test, predictions)
print(f'Root Mean Squared Error: {rmse}')
 
r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   23.6s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   57.3s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  1.5min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  2.3min


Mean Squared Error: 453529.6704072871
Mean Absolute Error: 215.74984052776685
Root Mean Squared Error: 545.5143559310047
R-squared: 0.995822081465356


## Gradient boosting

In [6]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [8]:
regr_I = GradientBoostingRegressor(random_state=1, max_depth=20, 
                                min_samples_split=170, n_estimators=230*2, 
                                learning_rate=0.2/2)

In [9]:
regr_I.fit(X_train, y_train.iloc[:,0])

GradientBoostingRegressor(max_depth=20, min_samples_split=170, n_estimators=460,
                          random_state=1)

Mean Squared Error: 147154.3059721577
Mean Absolute Error: 178.9607111962562
Root Mean Squared Error: 383.60696809645896
R-squared: 0.9999407979177086

In [10]:
predictions = regr_I.predict(X_test)
 
# Evaluating the model
mse = mean_squared_error(y_test.iloc[:,0], predictions)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(y_test.iloc[:,0], predictions)
print(f'Mean Absolute Error: {mae}')

rmse = root_mean_squared_error(y_test.iloc[:,0], predictions)
print(f'Root Mean Squared Error: {rmse}')
 
r2 = r2_score(y_test.iloc[:,0], predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 147154.3059721577
Mean Absolute Error: 178.9607111962562
Root Mean Squared Error: 383.60696809645896
R-squared: 0.9999407979177086


In [30]:
regr_Q = GradientBoostingRegressor(random_state=1, max_depth=20, 
                                min_samples_split=170, n_estimators=230*2, 
                                learning_rate=0.2/2, verbose=2)

In [31]:
regr_Q.fit(X_train, y_train.iloc[:,1])

      Iter       Train Loss   Remaining Time 
         1     2338277.8978           11.45m
         2     1904294.4350           11.15m
         3     1551730.8735           11.28m
         4     1264993.4525           11.27m
         5     1031992.2246           11.25m
         6      842421.9931           11.23m
         7      688233.7132           11.21m
         8      562511.8520           11.20m
         9      460019.3816           11.19m
        10      376625.5335           11.19m
        11      308446.0634           11.18m
        12      252905.8238           11.15m
        13      207578.9271           11.13m
        14      170577.0611           11.12m
        15      140353.4520           11.09m
        16      115572.9957           11.06m
        17       95358.7922           11.02m
        18       78793.6693           11.06m
        19       65198.6108           11.04m
        20       54042.9039           11.03m
        21       44939.4265           11.07m
        2

GradientBoostingRegressor(max_depth=20, min_samples_split=170, n_estimators=460,
                          random_state=1, verbose=2)

In [34]:
predictions = regr_Q.predict(X_test)
 
# Evaluating the model
mse = mean_squared_error(y_test.iloc[:,1], predictions)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(y_test.iloc[:,1], predictions)
print(f'Mean Absolute Error: {mae}')

rmse = root_mean_squared_error(y_test.iloc[:,1], predictions)
print(f'Root Mean Squared Error: {rmse}')
 
r2 = r2_score(y_test.iloc[:,1], predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 317.84422753720355
Mean Absolute Error: 8.348236262827204
Root Mean Squared Error: 17.8281863221474
R-squared: 0.9998879124399651


In [13]:
with open('Models/gradient-boosting-I-1.pkl','wb') as f:
    pickle.dump(regr_I,f)

In [37]:
with open('Models/gradient-boosting-Q-1.pkl','wb') as f:
    pickle.dump(regr_Q,f)

## Gradient boosting model time evaluation

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import pickle
import time

In [ ]:
# regr_I = GradientBoostingRegressor(random_state=1, max_depth=20, 
#                                 min_samples_split=170, n_estimators=230*2, 
#                                 learning_rate=0.2/2)

In [4]:
regr_I = pickle.load(open("Models/gradient-boosting-I-1.pkl", "rb"))

In [5]:
regr_I

GradientBoostingRegressor(max_depth=20, min_samples_split=170, n_estimators=460,
                          random_state=1)

In [13]:
start_time = time.time()
predictions = regr_I.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))
 
# Evaluating the model
mse = mean_squared_error(y_test.iloc[:,0], predictions)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(y_test.iloc[:,0], predictions)
print(f'Mean Absolute Error: {mae}')

rmse = root_mean_squared_error(y_test.iloc[:,0], predictions)
print(f'Root Mean Squared Error: {rmse}')
 
r2 = r2_score(y_test.iloc[:,0], predictions)
print(f'R-squared: {r2}')

--- 16.738155603408813 seconds ---
Mean Squared Error: 147154.3059721577
Mean Absolute Error: 178.9607111962562
Root Mean Squared Error: 383.60696809645896
R-squared: 0.9999407979177086


In [14]:
X_test

,0,1,2,3,4
406119,92.0,0.0014,2.2,0.011091,0.015920
488368,100.0,0.0030,0.6,0.003427,0.806686
500225,104.0,0.0016,1.4,0.000327,0.337873
1233605,200.0,0.0014,0.9,0.158245,0.337873
1049759,176.0,0.0014,1.2,1.875661,0.337873
...,...,...,...,...,...
1081567,180.0,0.0014,2.5,0.008661,0.806686
982847,168.0,0.0010,1.3,0.024772,0.337873
695529,128.0,0.0024,1.4,0.000504,0.015920
91744,48.0,0.0030,3.2,0.006359,0.806686
